In [1]:
import numpy as np
import numpy.linalg as la
from sympy import *
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [3]:
a = -8
b = 11
i = 4

r = 0.618**i*(b-a)
r

2.7714528937439997

In [9]:
# 16.2
x = Symbol('x')
h = Symbol('h')
x0 = 1
f = 1*x**3 + 2*x**2 + 3*x +4
approx = f.subs(x,x0) + diff(f,x).subs(x,x0)*h + 0.5 * diff(f,x,2).subs(x,x0)*h**2
approx

5.0*h**2 + 10*h + 10

In [12]:
# 16.4
x0 = 0.25
x = Symbol('x')
f = -exp(-x**2)
x1 = x0 - diff(f,x).subs(x,x0)/diff(f,x,2).subs(x,x0)
x1

-0.0357142857142857

In [11]:
# 16.10
def hessian(f, X):
    x0, y0 = X[0, 0], X[1,0]
    x, y = Symbol('x'), Symbol('y')
    A = Matrix([f])
    B = Matrix([x, y])
    Hessian = A.jacobian(B).jacobian(B).subs({x:x0, y:y0})
    return np.array(Hessian).astype(np.float64)

def gradient(f, X):
    x0, y0 = X[0, 0], X[1,0]
    x, y = Symbol('x'), Symbol('y')
    A = Matrix([f])
    B = Matrix([x, y])
    Gradient = A.jacobian(B).subs({x:x0, y:y0})
    return np.array(Gradient).astype(np.float64).T

def newtons_method(f, x_init, tol):
    x_new = x_init
    x_prev = np.random.randn(x_init.shape[0])
    cnt = 0
    while(la.norm(gradient(f, x_new)) > tol):
        x_prev = x_new
        print(x_prev)
        s = -la.solve(hessian(f, x_prev), gradient(f, x_prev))
        x_new = x_prev + s
        print(x_new)
        cnt += 1
    return x_new, cnt

In [12]:
x_init = np.array([[1],[2]])
x, y = Symbol('x'), Symbol('y')
f = 22 * x**2- 5 * x + 5 + 20*y**2- x*y
tol = 1e-10
newtons_method(f, x_init, tol)

[[1]
 [2]]
[[0.11370097]
 [0.00284252]]


(array([[0.11370097],
        [0.00284252]]),
 1)

In [83]:
# 16.11
x, y = Symbol('x'), Symbol('y')
f = 1.5*x**2 + 2*x*y + 2.5*y**2
X0 = np.array([[8], [5]])
#s0 = -la.solve(hessian(f, X0), gradient(f, X0).T)
s0 = -X0
s0

array([[-8],
       [-5]])

In [108]:
# 16.12
x, y = Symbol('x'), Symbol('y')
f = 7*x**2 + 3*x*y + 7*y**2 + 13*exp(10*x*y) + 14*sin(y)**2 + 2*cos(x*y)
x0 = np.array([[0], [1]])
print(gradient(f, x0))
print()
print(hessian(f, x0))
s1 = -la.solve(hessian(f, x0), gradient(f, x0))
x1 = x0 + s1
print(x1)
print()
print(-gradient(f, x0))

[[133.        ]
 [ 26.73016398]]

[[1312.          133.        ]
 [ 133.            2.34788858]]
[[-0.22198221]
 [ 2.18977935]]

[[-133.        ]
 [ -26.73016398]]


In [107]:
# 16.13
x, y = Symbol('x'), Symbol('y')
f = 3 + x**2/8 + y**2/8 - sin(x)*cos(sqrt(2)/2*y)
x0 = np.array([[pi/3], [pi/2/sqrt(2)]]).astype(np.float64)
alpha = 1.581
print(gradient(f, x0))
print()
print(hessian(f, x0))
s1 = -la.solve(hessian(f, x0), gradient(f, x0))
x1 = x0 + s1
print(x1)
print()
s2 = -gradient(f, x0)
x2 = x0 + alpha * s2
print(x2)

[[-0.091754  ]
 [ 0.71069289]]

[[0.86237244 0.25      ]
 [0.25       0.55618622]]
[[ 1.59546844]
 [-0.41351805]]

[[ 1.19226063]
 [-0.01288472]]


In [ ]:
# 16.14
import numpy as np
import matplotlib.pyplot as plt

brackets = []
gs = (np.sqrt(5) - 1) / 2
m1 = a + (1 - gs) * (b - a)
m2 = a + gs * (b - a)

# Begin your modifications below here
f1, f2 = f(m1), f(m2)
while b - a > 1e-5:
    if f1>= f2:
        a = m1
    else:
        b = m2
    m1 = a + (1 - gs) * (b - a)
    m2 = a + gs * (b - a)    
    if f1>= f2:
        f1 = f2
        f2 = f(m2)
    else:
        f2 = f1
        f1 = f(m1)
    
    brackets.append([a, m1, m2, b])

# End your modifications above here

# Plotting code below, no need to modify
x = np.linspace(-10, 10)
plt.plot(x, f(x))

brackets = np.array(brackets)
names=['a', 'm1', 'm2', 'b']
for i in range(4):
    plt.plot(brackets[:, i], 3*np.arange(len(brackets)), 'o-', label=names[i])
plt.legend()

In [10]:
def func(r):
    x, y = r
    return np.float64(3 +((x**2)/8) + ((y**2)/8) - np.sin(x)*np.cos((2**-0.5)*y))

def obj_func(alpha, x, s):
    # code for computing the objective function at (x+alpha*s)
    return func(x + alpha * s)

def steepest_descent(f, x_init, tol):
    x_new = x_init
    x_prev = np.random.randn(x_init.shape[0])
    while(la.norm(x_prev - x_new, 2) > tol):
        x_prev = x_new
        s = -gradient(f, x_prev)
        alpha = opt.minimize_scalar(obj_func, args=(x_prev, s)).x
        x_new = x_prev + alpha*s

    return x_new

In [11]:
x, y = Symbol('x'), Symbol('y')
f = 3 + x**2/8 + y**2/8 - sin(x)*cos(sqrt(2)/2*y)
x_init = np.array([[pi/3], [pi/2/sqrt(2)]]).astype(np.float64)
x_new = x_init
x_prev = np.random.randn(x_init.shape[0])
while(la.norm(x_prev - x_new, 2) > 1e-6):
    x_prev = x_new
    
    s = -gradient(f, x_prev)
    print('s:%s'%s)
    alpha = np.float64(opt.minimize_scalar(obj_func, args=(x_prev, s)).x)
    print('alpha:%s'%alpha)
    x_new = x_prev + alpha*s
    print('x_new:%s'%x_new)

s:[[ 0.091754  ]
 [-0.71069289]]
alpha:1.5810402265646062
x_new:[[ 1.19226432]
 [-0.01291331]]
s:[[0.07147531]
 [0.00922782]]
alpha:0.8462419361003177
x_new:[[ 1.25274973]
 [-0.00510434]]
s:[[-0.00047774]
 [ 0.00370025]]
alpha:1.3647102624709635
x_new:[[ 1.25209775e+00]
 [-5.45660557e-05]]
s:[[3.06504156e-04]
 [3.95506775e-05]]
alpha:0.8389914118301472
x_new:[[ 1.25235490e+00]
 [-2.13833769e-05]]
s:[[-1.99968171e-06]
 [ 1.55000031e-05]]
alpha:1.364933486946003
x_new:[[ 1.25235217e+00]
 [-2.26903591e-07]]
s:[[1.27492010e-06]
 [1.64473755e-07]]
alpha:0.8384790778699812
x_new:[[ 1.25235324e+00]
 [-8.89957885e-08]]
s:[[-7.57729302e-09]
 [ 6.45096662e-08]]
alpha:1.579814012086196
x_new:[[1.25235323e+00]
 [1.29174860e-08]]


In [122]:
import numpy as np
from sympy import *
import numpy.linalg as la
import scipy.optimize as opt
import matplotlib.pyplot as plt

def func(r):
    x, y = r
    return np.float64(3 +((x**2)/8) + ((y**2)/8) - np.sin(x)*np.cos((2**-0.5)*y))

def obj_func(alpha, x, s):
    # code for computing the objective function at (x+alpha*s)
    return func(x + alpha * s)

def steepest_descent(f, x_init, tol):
    x_new = x_init
    x_prev = np.random.randn(x_init.shape[0])
    cnt = -1
    error = la.norm(x_prev - x_new, 2)
    xs = [x_new]
    while(error > tol):
        x_prev = x_new
        s = -gradient(f, x_prev)
        alpha = opt.minimize_scalar(obj_func, args=(x_prev, s)).x
        x_new = x_prev + alpha*s
        cnt += 1
        error = la.norm(x_prev - x_new, 2)
        xs.append(x_new)
        
    return x_new.flatten(), cnt, np.array(xs).reshape(-1, 2)

def hessian(f, X):
    x0, y0 = X[0, 0], X[1,0]
    x, y = Symbol('x'), Symbol('y')
    A = Matrix([f])
    B = Matrix([x, y])
    Hessian = A.jacobian(B).jacobian(B).subs({x:x0, y:y0})
    return np.array(Hessian).astype(np.float64)

def gradient(f, X):
    x0, y0 = X[0, 0], X[1,0]
    x, y = Symbol('x'), Symbol('y')
    A = Matrix([f])
    B = Matrix([x, y])
    Gradient = A.jacobian(B).subs({x:x0, y:y0})
    return np.array(Gradient).astype(np.float64).T

def newtons_method(f, x_init, tol):
    x_new = x_init
    x_prev = np.random.randn(x_init.shape[0])
    cnt = -1
    error = la.norm(x_prev - x_new, 2)
    xs = [x_new]
    while(error > tol):
        x_prev = x_new
        s = -la.solve(hessian(f, x_prev), gradient(f, x_prev))
        x_new = x_prev + s
        cnt += 1
        error = la.norm(x_prev - x_new, 2)
        xs.append(x_new)
    return x_new.flatten(), cnt, np.array(xs).reshape(-1, 2)

x, y = Symbol('x'), Symbol('y')
f = 3 + x**2/8 + y**2/8 - sin(x)*cos(sqrt(2)/2*y)
x_init = r_init.reshape((2,1))
r_newton, iteration_count_newton, rs_newton = newtons_method(f, x_init, stop)
r_sd, iteration_count_sd, rs_sd = steepest_descent(f, x_init, stop)
print(r_newton, r_sd, rs_newton)
print(rs_newton.reshape(-1, 2))
errors_newton = rs_newton - r_newton.reshape(1, 2)
errors_newton = errors_newton[:-1,:]
print(errors_newton, iteration_count_newton)
newton_list= []
for i in range(errors_newton.shape[0]):
    newton_list.append(np.log(la.norm(errors_newton[i,:], 2)))
print(newton_list)

errors_sd = rs_sd - r_sd.reshape(1, 2)
errors_sd = errors_sd[:-1,:]
sd_list= []
for i in range(errors_sd.shape[0]):
    sd_list.append(np.log(la.norm(errors_sd[i,:], 2)))
plt.plot(np.arange(iteration_count_newton + 1), np.array(newton_list), label = 'Newton method line')
plt.plot(np.arange(iteration_count_sd + 1), np.array(sd_list), label = 'Steepest descent line')
plt.legend()
plt.title('Error vs. Iterations')
plt.xlabel('Iteration count')
plt.ylabel('Error')



2.6789179719752454

In [ ]:
# 16.16
import numpy as np
from sympy import *

# complete the function below
def dfunc(x):
    # Add your code here
    y = Symbol('y')
    f = -exp(-y**2) * (y + sin(y))
    return diff(f,y).subs(y,x)
    

# complete the function below
def d2func(x):
    # Add your code here
    y = Symbol('y')
    f = -exp(-y**2) * (y + sin(y))
    return diff(f,y,2).subs(y,x)

# run Newton's Method
newton_list = [x0]
x = x0
while abs(dfunc(x)) > tol:
    x -= dfunc(x)/d2func(x)
    newton_list.append(x)
newton_guesses = np.array(newton_list).astype(np.float64)

In [ ]:
# HW 16.15

from math import sin, cos
import numpy as np
import numpy.linalg as la
import sympy as sp
import scipy.optimize as opt
import matplotlib.pyplot as plt

def f(alpha, r, s):
    r += s*alpha
    x, y = r
    return 3 +((x**2)/8) + ((y**2)/8) - np.sin(x)*np.cos((2**-0.5)*y)

def sd_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    alpha = opt.minimize_scalar(f, args=(x_prev, neg_gradient_at)).x
    return np.array(x_prev) + alpha * neg_gradient_at, la.norm(neg_gradient_at, 2)

def gradient(formula, symbols, values=None):
    '''
    Given a SymPy formula and variables
    Find its analytic gradient without substituion
    as a list of SymPy formulae or numerical gradient
    if values specified
    '''
    size = len(symbols)
    gradient = []
    for i in range(size):
        gradient.append(formula.diff(symbols[i]))
        
    if values == None:
        return gradient

    # Make sure you don't mess up the analytical gradient
    g_copy = gradient.copy()
    gradient_at = []
    for i in range(len(g_copy)):
        for j in range(len(symbols)):
            g_copy[i] = g_copy[i].subs(symbols[j], values[j])
        gradient_at.append(float(g_copy[i].evalf()))
    return gradient_at

def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())

def hessian(gradient, symbols, values=None):
    '''
    Given an analytic gradient and variables
    Calculate its analytic Hessian
    or numerical Hessian if values are specified
    '''
    size = len(symbols)
    hessian = []
    for i in range(size):
        hessian.append([0]*size)
    for i in range(size):
        for j in range(size):
            hessian[i][j] = gradient[i].diff(symbols[j])
            
    if values == None:
        return hessian
    
    hessian_at = hessian.copy()
    size = len(hessian)
    for i in range(size):
        for j in range(size):
            hessian_at[i][j] = subs_all(hessian_at[i][j], symbols, [float(v) for v in values])
#             for k in range(len(symbols)):
#                 hessian_at[i][j] = hessian_at[i][j].subs(symbols[k], values[k])
#             hessian_at[i][j] = float(hessian_at[i][j].evalf())
    return hessian_at

def newton_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    grad = gradient(formula, symbols)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    hes_at = np.array(hessian(grad, symbols, values=x_prev))
    return np.array(x_prev) + la.solve(hes_at, neg_gradient_at), la.norm(neg_gradient_at, 2)

def newton_nd_optimization_crude(f_str, s_str, start, tolerance):
    '''
    A crude version of Newton ND Optimization algorithm
    Maybe you can help out implementing an analytic solution
    Returning the numerical solution as well as the number of
    iterations it took
    '''
    x_list = []
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    gradient = np.inf
    while (gradient > tolerance):
        x_list.append(curr)
        curr, gradient = newton_nd_optimization_crude_step(formula, symbols, curr)
        _, gradient = newton_nd_optimization_crude_step(formula, symbols, curr)
        iterations += 1
    x_list.append(curr)
    return curr, iterations, x_list

def sd_nd_optimization_crude(f_str, s_str, start, tolerance):
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    gradient = np.inf
    x_list = []
    while (gradient > tolerance):
        x_list.append(curr)
        curr, gradient = sd_nd_optimization_crude_step(formula, symbols, curr)
        iterations += 1
    x_list.append(curr)
    return curr, iterations, x_list


r_newton, iteration_count_newton, newton_list = newton_nd_optimization_crude(f_str = '3 +((x**2)/8) + ((y**2)/8) - sin(x)*cos((2**-0.5)*y)', 
                                                                s_str = 'x y', 
                                                                start = r_init, 
                                                                tolerance = stop)

r_sd, iteration_count_sd, sd_list = sd_nd_optimization_crude(f_str = '3 +((x**2)/8) + ((y**2)/8) - sin(x)*cos((2**-0.5)*y)', 
                                                    s_str = 'x y', 
                                                    start = r_init, 
                                                    tolerance = stop*10)


newton_plot = [np.log(la.norm(r-r_newton,2)) for r in newton_list]
sd_plot = [np.log(la.norm(r-r_sd,2)) for r in sd_list]

plt.plot(sd_plot, label='Steepest descent line')
plt.plot(newton_plot, label='newton')

plt.xlabel('Iteration')
plt.ylabel('log(Error)')
plt.legend()
plt.title('Error vs. Iteration')